# 🔍 Enterprise RAG File Search with Azure AI Foundry (.NET)

## 📋 Advanced RAG Implementation Tutorial

This notebook demonstrates building a production-ready **Retrieval-Augmented Generation (RAG)** system using the Microsoft Agent Framework for .NET and Azure AI Foundry. You'll create an enterprise-grade document search and knowledge retrieval system with advanced AI capabilities.

## 🎯 Learning Objectives

### 🏗️ **Enterprise RAG Architecture**
- **Document Ingestion**: Upload and process various document formats
- **Vector Search**: Semantic similarity search across large document collections
- **Context Integration**: Combine retrieved information with AI reasoning
- **Persistent Storage**: Enterprise-grade document and vector storage

### 🔍 **Advanced Search Capabilities**
- **Multi-Modal Search**: Text, images, and structured data retrieval
- **Semantic Understanding**: Context-aware document matching
- **Relevance Ranking**: Intelligent result prioritization
- **Real-Time Updates**: Dynamic document collection management

### 🏢 **Production RAG Features**
- **Azure Integration**: Native Azure AI Foundry and Cognitive Services
- **Scalable Storage**: Enterprise document and vector databases
- **Security & Compliance**: Azure identity and access management
- **Performance Optimization**: Efficient retrieval and processing patterns

## ⚙️ Prerequisites & Setup

### 📦 **Required NuGet Packages**

This implementation uses the latest .NET AI ecosystem packages:

- **Microsoft.Extensions.AI 9.9.0**: Core AI abstractions and interfaces
- **Azure.AI.Agents.Persistent 1.2.0-beta.5**: Persistent agent capabilities with Azure
- **Azure.Identity 1.15.0**: Enterprise Azure authentication
- **System.Linq.Async 6.0.3**: Asynchronous LINQ operations for efficient data processing

### 🔑 **Azure Configuration Requirements**

**Required Azure Resources:**
- Azure AI Foundry workspace
- Azure Storage Account (for document storage)

**Authentication Setup:**
```bash
# Azure CLI authentication
az login
az account set --subscription "your-subscription-id"
azd auth
```

## 🏗️ **RAG System Architecture**

### Enterprise RAG Pipeline
```mermaid
graph TD
    A[Documents] --> B[Document Processing]
    B --> C[Vector Embedding]
    C --> D[Vector Database]
    
    E[User Query] --> F[Query Embedding] 
    F --> G[Similarity Search]
    D --> G
    G --> H[Context Retrieval]
    H --> I[AI Agent]
    I --> J[Enhanced Response]
```

**Key Components:**
- **Document Processor**: Parse and chunk documents for optimal retrieval
- **Vector Store**: High-performance semantic search capabilities
- **AI Agent**: Context-aware response generation
- **Search Integration**: Seamless query-to-response pipeline

## 🏢 **Enterprise Use Cases**

### 📚 **Knowledge Management**
- **Corporate Documentation**: Internal knowledge base search
- **Policy & Procedures**: Compliance and operational guidance
- **Training Materials**: Educational content retrieval
- **Technical Documentation**: Engineering and product information

### 🔍 **Customer Support**
- **FAQ Automation**: Intelligent support ticket resolution
- **Product Information**: Detailed product knowledge access
- **Troubleshooting**: Technical issue diagnosis and resolution
- **Documentation Search**: Customer-facing information retrieval

### 📊 **Business Intelligence**
- **Report Analysis**: Historical report and data analysis
- **Market Research**: Industry insights and competitive intelligence
- **Regulatory Compliance**: Legal and compliance document search
- **Decision Support**: Data-driven decision making assistance

Let's build an enterprise RAG system! 🚀

In [ ]:
// 📦 Core AI Framework Package
// Microsoft Extensions AI provides unified abstractions for AI services in .NET
// Essential foundation for building enterprise AI applications
#r "nuget: Microsoft.Extensions.AI, 9.9.0"

In [ ]:
// 🏢 Enterprise Azure AI and Supporting Packages
// Advanced Azure AI capabilities and utilities for production RAG systems

#r "nuget: Azure.AI.Agents.Persistent, 1.2.0-beta.5"  // 🔄 Persistent agent capabilities with state management
#r "nuget: Azure.Identity, 1.15.0"                     // 🔐 Enterprise Azure authentication and identity
#r "nuget: System.Linq.Async, 6.0.3"                  // ⚡ Asynchronous LINQ for efficient data processing

In [ ]:
// 🔗 Local Reference: Azure AI Agent Framework Integration
// Provides Azure-specific agent capabilities and Azure AI Foundry integration
// Contains enterprise-grade Azure AI service implementations
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI.AzureAI\bin\Debug\net9.0\Microsoft.Agents.AI.AzureAI.dll"

In [ ]:
// 🏗️ Local Reference: Core Microsoft Agent Framework
// Fundamental agent abstractions, base classes, and orchestration logic
// Essential building blocks for creating intelligent AI agents
#r "C:\Users\kinfeylo\Documents\Agent\agent-framework\dotnet\src\Microsoft.Agents.AI\bin\Debug\net9.0\Microsoft.Agents.AI.dll"

In [ ]:
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [ ]:
#r "nuget: DotNetEnv, 3.1.1"

In [ ]:
using System;
using System.Linq;
using Azure.AI.Agents.Persistent;
using Azure.Identity;
using Microsoft.Agents.AI;

In [ ]:
 using DotNetEnv;

In [ ]:
Env.Load("../../../.env");

In [ ]:
var azure_foundry_endpoint = Environment.GetEnvironmentVariable("FOUNDRY_PROJECT_ENDPOINT") ?? throw new InvalidOperationException("AZURE_FOUNDRY_PROJECT_ENDPOINT is not set.");
var azure_foundry_model_id = Environment.GetEnvironmentVariable("FOUNDRY_MODEL_DEPLOYMENT_NAME") ?? "gpt-4.1-mini";

In [ ]:
string pdfPath = "../files/demo.md";

In [ ]:
using System.IO;

async Task<Stream> OpenImageStreamAsync(string path)
{
	return await Task.Run(() => File.OpenRead(path));
}

var pdfStream = await OpenImageStreamAsync(pdfPath);

In [ ]:
var persistentAgentsClient = new PersistentAgentsClient(azure_foundry_endpoint, new AzureCliCredential());

In [ ]:
PersistentAgentFileInfo fileInfo = await persistentAgentsClient.Files.UploadFileAsync(pdfStream, PersistentAgentFilePurpose.Agents, "demo.md");

In [ ]:
PersistentAgentsVectorStore fileStore =
            await persistentAgentsClient.VectorStores.CreateVectorStoreAsync(
                [fileInfo.Id],
                metadata: new Dictionary<string, string>() { { "agentkey", bool.TrueString } });

In [ ]:
PersistentAgent agentModel = await persistentAgentsClient.Administration.CreateAgentAsync(
            azure_foundry_model_id,
            name: "DotNetRAGAgent",
            tools: [new FileSearchToolDefinition()],
            instructions: """
                You are a helpful assistant that helps people find information in a set of files.  If you can't find the answer in the files, just say you don't know. Do not make up an answer.
                """,
            toolResources: new()
            {
                FileSearch = new()
                {
                    VectorStoreIds = { fileStore.Id },
                }
            },
            metadata: new Dictionary<string, string>() { { "agentkey", bool.TrueString } });

In [ ]:
AIAgent agent = await persistentAgentsClient.GetAIAgentAsync(agentModel.Id);

In [ ]:
AgentThread thread = agent.GetNewThread();

In [ ]:
Console.WriteLine(await agent.RunAsync("What's graphrag?", thread));